In [1]:
from post_parser_record import PostParserRecord
post_reader = PostParserRecord("Posts_Coffee.xml")
!pip3 install nltk
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nicdr_nactbrj\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nicdr_nactbrj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Saves nessisary information from documents for calcumating TF-IDF

In [2]:
#creating index
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = stopwords.words('english')
stop_words.append('p')

inverted_index = {}
word_count_index = {} #{document_id,word_count}
total_document_count = 0
#function to update the index needed for TF-IDF
def update_index(tokens, document_id):
  global total_document_count
  total_document_count += 1
  
  word_count_index[document_id] = len(tokens)
  #runs through all tokens and indexed accordingly
  for token in tokens:
    #if token not in stopwords.words('english'):
      if token in inverted_index:
        if document_id in inverted_index[token]:
          inverted_index[token][document_id] += 1
        else:
          inverted_index[token][document_id] = 1
      else:
        inverted_index[token] = {document_id: 1}

#functions to create tokens
def create_tokens_for_text(text):
  temp_tokens = word_tokenize(re.sub(r'[^\w\s]', ' ', text))
  tokens = [i for i in temp_tokens if i not in stop_words]
  return tokens

#indexes questions
print("indexing questions...")
for question_id in post_reader.map_questions:
  question = post_reader.map_questions[question_id]
  title = question.title.lower().strip()
  body = question.body.lower().strip()
  text = title + " " + body
  tokens = create_tokens_for_text(text)
  update_index(tokens, question_id)
  
print("complete")

#indexes answers
print("indexing answers...")
for answer_id in post_reader.map_just_answers:
  answer = post_reader.map_just_answers[answer_id]
  text = answer.body.lower().strip()
  tokens = create_tokens_for_text(text)
  update_index(tokens, answer_id)
  
print("Complete")


indexing questions...
complete
indexing answers...
Complete


Calculates TF-IDF for chosen querys 

In [3]:
  
#run to simply initalize this function for use 
from re import A
import math
#searches index for document matches
def TFIDF(query,query_num):
  tfidf = {}
  tf = 0
  for word in query:
    total_word_count = 0
    #counts total words
    for document in inverted_index[word]:
      total_word_count += inverted_index[word][document]
    #continues
    for document in inverted_index[word]:
      if document not in tfidf:
        tfidf[document] = (inverted_index[word][document]/word_count_index[document]) * (math.log(((1+total_document_count)/(1+total_word_count))+1))
      else:
        tfidf[document] += (inverted_index[word][document]/word_count_index[document]) * (math.log(((1+total_document_count)/(1+total_word_count))+1))
  result = sorted(tfidf.items(), key=lambda x: x[1], reverse=True)
  i = 0
  for items in result:
    #makes sure only 5 of each are printed
    if i <= 4:
      print("Document: "+str(items[0])+"\tScore: "+str(items[1]))
      i += 1
    else:
      break
################################################################################
#Querys
querys = ["espresso","turkish coffee","making a decaffeinated coffee","can I use the same coffee grounds twice?"]

query_num = 0
#scans through and prints each query
for temp_query in querys:
  temp_query = temp_query.lower().strip()
  temp_tokens = word_tokenize(re.sub(r'[^\w\s]', '', temp_query))
  query_tokens = [i for i in temp_tokens if i not in stop_words]

  print("Searching: "+temp_query)
  #uses function to search index and print final result
  TFIDF(query_tokens,query_num)
  query_num += 1
  

Searching: espresso
Document: 4404   Score: 0.21116651151731888
Document: 4175   Score: 0.18204009613561972
Document: 2867   Score: 0.17597209293109906
Document: 3168   Score: 0.17597209293109906
Document: 3904   Score: 0.17597209293109906
Searching: turkish coffee
Document: 5094   Score: 0.7134110891217438
Document: 5182   Score: 0.6153617051246203
Document: 2522   Score: 0.6089518241789702
Document: 483   Score: 0.4733551577881695
Document: 209   Score: 0.4102411367497469
Searching: making a decaffeinated coffee
Document: 204   Score: 0.7342729167619662
Document: 3293   Score: 0.46727202514212884
Document: 120   Score: 0.4213748516426526
Document: 2897   Score: 0.4005125000519816
Document: 373   Score: 0.26701258579550213
Searching: can i use the same coffee grounds twice?
Document: 2683   Score: 0.5339452613665286
Document: 1749   Score: 0.36111912934403323
Document: 3258   Score: 0.318020209424038
Document: 3966   Score: 0.30179366458577406
Document: 3818   Score: 0.243603497989281